## Feature Engineering & Justification

### DATA CLEANING

Missing values in the latitude and longitude columns will be filled using the geopy library, which can convert a place (like a county) into its corresponding latitude and longitude.

In [ ]:
nom = ArcGIS()
def get_coordinates(county_name):
    try:
        location = nom.geocode(county_name)
        if location:
            return location.latitude, location.longitude
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Error geocoding {county_name}: {e}")
    return None, None

In [ ]:
for index, row in stormEvents_2014.iterrows():
    if pd.isna(row['BEGIN_LAT']) or pd.isna(row['BEGIN_LON']):
        lat, lon = get_coordinates(row['CZ_NAME'])
        if lat is not None and lon is not None:
            stormEvents_2014.at[index, 'BEGIN_LAT'] = lat
            stormEvents_2014.at[index, 'BEGIN_LON'] = lon
            stormEvents_2014.at[index, 'END_LAT'] = lat
            stormEvents_2014.at[index, 'END_LON'] = lon

In [ ]:
stormEvents_2014[['CZ_NAME', 'BEGIN_LAT', 'BEGIN_LON']]